import 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from itertools import product
from Utils.dataManagingUtils import save_prediction_to_file, save_data_to_csv, print_results, print_best_results, searchMaxRowByColumn
from Utils.transformersUtils import OryginalData, PCATransformer, CORRTransformer, LassoSelector, RFCSelector

In [ ]:
# Wartości zmiennych 
n_jobs=-1
n_iter_randomSearch_rf=1000
n_iter_randomSearch_sv=1000
n_iter_randomSearch_gb=1000
n_iter_randomSearch_dt=10000
n_iter_randomSearch_final = 5000
verbose=1
random_state=42
cv=7
numberOfPoints=25

# Wartości hiperparametrów
n_estimators_lower_rf = 5
n_estimators_upper_rf = 100
max_depth_lower_rf = 1
max_depth_upper_rf = 10
min_samples_split_lower_rf = 1
min_samples_split_upper_rf = 5
min_samples_leaf_lower_rf = 1
min_samples_leaf_upper_rf = 10
max_features_rf = ['sqrt', 'log2']

cost_lower_sv = 0.01
cost_upper_sv = 10.0
kernel_sv = ['rbf', 'sigmoid']
gamma_lower_sv = 0.001
gamma_upper_sv = 6.5

n_estimators_lower_gb = 5
n_estimators_upper_gb = 100
max_depth_lower_gb = 1
max_depth_upper_gb = 8
min_samples_split_lower_gb = 1
min_samples_split_upper_gb = 10
min_samples_leaf_lower_gb = 1
min_samples_leaf_upper_gb = 5
learning_rate_lower_gb = 0.05
learning_rate_upper_gb = 0.5
subsample_lower_gb = 0.75
subsample_upper_gb = 1.0

max_depth_lower_dt = 1
max_depth_upper_dt = 10
min_samples_split_lower_dt = 1
min_samples_split_upper_dt = 5
min_samples_leaf_lower_dt = 1
min_samples_leaf_upper_dt = 10
max_features_dt = ['sqrt', 'log2']

n_neighbors_lower = 1
n_neighbors_upper = 20
weights =  ['uniform', 'distance']
p = [1, 2]

In [ ]:
X_train = pd.read_csv("../Dane/Oryginalne/artificial_train.data", sep=" ", header=None).iloc[:,:-1]
y_train = pd.read_csv("../Dane/Oryginalne/artificial_train_labels.data", sep=' ', header=None).squeeze(axis=1)
X_test = pd.read_csv("../Dane/Oryginalne/artificial_test.data", sep=" ", header=None).iloc[:,:-1]

Ponieważ gridsearch miał problem odczytywaniem transformerów z innego pliku .ipynb zdecydowano się zdefiniować transfromery w pliku pythonowym. Transformery są zgodne z poczynionymi rozważeniami w pliku preprocessing.

In [ ]:
transformers = [
    OryginalData(),
    PCATransformer(0.3),
    PCATransformer(0.4),
    PCATransformer(0.6),
    PCATransformer(0.8),
    CORRTransformer(0.05),
    CORRTransformer(0.075),
    CORRTransformer(0.7),
    LassoSelector(),
    RFCSelector(),
]

# Etap 1
Poniżej znajduje kod pozwalający uzyskać przesłanki do wyboru odpowiedniego stacka algorytmów uczenia maszynowego. Ponieważ zbiór danych jest dość mały to zdecydowano się na cv = 7. Ponadto wybrano hiperparametry w taki sposób aby zminimalizować ryzyko overfittingu, a żeby oddać realnie potencjał znajdujący się w poszczególnych kombinacjach algorytmów uczenia maszynowego oraz metodzie preprocessingu. 

In [ ]:
# 1h 15min
model_list_1 = [
    ('rf1', RandomForestClassifier(n_estimators=15, max_depth=2, random_state=42)),
    ('gb1', GradientBoostingClassifier(n_estimators=15, max_depth=2,learning_rate=0.1, random_state=42)),
    ('sv1', SVC(C=1.0, kernel='rbf', gamma='scale', random_state=42)),
    ('kn1', KNeighborsClassifier(n_neighbors=5)),
    ('dt1', DecisionTreeClassifier(max_depth=2, random_state=42)),
    ('mlp1',MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50, ), random_state=42)),
]

model_list_2 = [
    ('rf2', RandomForestClassifier(n_estimators=15, max_depth=2, random_state=42)),
    ('gb2', GradientBoostingClassifier(n_estimators=15, max_depth=2, random_state=42)),
    ('sv2', SVC(C=1.0, kernel='rbf', gamma='scale', random_state=42)),
    ('kn2', KNeighborsClassifier(n_neighbors=5, )),
    ('dt2', DecisionTreeClassifier(max_depth=2, random_state=42)),
    ('mlp2', MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50, ), random_state=42)),
]

all_model_combinations = list(product(model_list_1, model_list_2))
formatted_model_combinations = [[(name1, model1), (name2, model2)] for ((name1, model1), (name2, model2)) in all_model_combinations]

for transformer in transformers:
    stack = StackingClassifier(estimators=[], final_estimator=LogisticRegression())
    pipeline = Pipeline([
        ('preprocessing', transformer),
        ('stack', stack)
    ])

    param_grid = {
        'stack__estimators':formatted_model_combinations
    }

    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        n_jobs=n_jobs,
        verbose=verbose,
        cv=cv,
        scoring='balanced_accuracy'
        )

    grid_search.fit(X_train, y_train)

    cs_results = pd.DataFrame(grid_search.cv_results_)
    save_data_to_csv(cs_results, f"../Wyniki/ModelSearch/Etap1/{type(transformer).__name__}.csv")


In [ ]:
analisys1_OryginalDataTransformer = pd.read_csv('../Wyniki/ModelSearch/Etap1/OryginalData.csv')
analisys1_PCA03Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap1/PCA03Transformer.csv')
analisys1_PCA04Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap1/PCA04Transformer.csv')
analisys1_PCA06Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap1/PCA06Transformer.csv')
analisys1_PCA08Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap1/PCA08Transformer.csv')
analisys1_CORR005Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap1/CORR005Transformer.csv')
analisys1_CORR075Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap1/CORR075Transformer.csv')
analisys1_CORR7Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap1/CORR7Transformer.csv')
analisys1_LassoTransformer = pd.read_csv('../Wyniki/ModelSearch/Etap1/LassoSelector.csv')
analisys1_RFCTransformer = pd.read_csv('../Wyniki/ModelSearch/Etap1/RFCSelector.csv')

In [ ]:
transformers = ["OryginalDataTransformer", "PCA03Transformer", "PCA04Transformer",
                "PCA06Transformer", "PCA08Transformer", "CORR005Transformer",
                "CORR075Transformer", "CORR7Transformer", "LassoTransformer",
                "RFCTransformer"]

analisys_list = [analisys1_OryginalDataTransformer, analisys1_PCA03Transformer, analisys1_PCA04Transformer,
                 analisys1_PCA06Transformer, analisys1_PCA08Transformer, analisys1_CORR005Transformer,
                 analisys1_CORR075Transformer, analisys1_CORR7Transformer, analisys1_LassoTransformer,
                 analisys1_RFCTransformer]

print_results(transformers, analisys_list)

Powyższe informacje miały na celu pokazać, które metody preprocessingu zawiodły. Stąd też w dalszych rozważaniach nie będziemy analizować PCA08Transformer(), CORR005Transformer(), CORR075Transformer(), CORR7Transformer(), LassoSelector(). Widać, że najlepsze rezultaty uzykano dla PCA04Transformer(), PCA03Transformer() i RFCSelector().

In [ ]:
print_best_results(analisys1_PCA03Transformer, "PCA03Transformer")
print_best_results(analisys1_PCA04Transformer, "PCA04Transformer")
print_best_results(analisys1_RFCTransformer, "RFCTransformer")
print_best_results(analisys1_OryginalDataTransformer, "OryginalDataTransformer")
print_best_results(analisys1_PCA06Transformer, "PCA06Transformer")

Z powyższych wyników dla 7 najlepszych wytrenowanych stacków można wysnuć następujące wnioski dla danych metod preprocessingu:

bests_PCA03Transformer:
- KNeighborsClassifier pojawił się we wszystkich stackach
- Najlepszy wynik uzyskano dla KNeighborsClassifier i SVC

bests_PCA04Transformer:
- SVC pojawił się we wszystkich stackach
- Najlepszy wynik uzyskano dla GradientBoostingClassifier i SVC

bests_RFCTransformer:
- KNeighborsClassifier pojawił się we wszystkich stackach
- Najlepszy wynik uzyskano dla KNeighborsClassifier i SVC

bests_OryginalDataTransformer:
- KNeighborsClassifier pojawił się we wszystkich stackach
- Najlepszy wynik uzyskano dla GradientBoostingClassifier i KNeighborsClassifier

bests_PCA06Transformer:
- SVC pojawił się we wszystkich stackach
- Najlepszy wynik uzyskano dla GradientBoostingClassifier i SVC

Różnice między wynikami konkretnych modeli w ramach jednego pliku są znikome. 
Biorąc pod uwagę powyższe wnioski i wyniki modelów z użyciem PCA zdecydowano się na ponownie przeprowadzenie wyszukiwania odpowiednego stacku, ale tym razem z mniejszą siatką modeli uczenia maszynowego i innymi transformatorami

Postanowiono znaleźć lokalne maksimum dla metody PCA: 

# Etap 2

In [ ]:
newTransformers = [
    PCATransformer(0.3),
    PCATransformer(0.4),
    RFCSelector(),
    PCATransformer(9),
    PCATransformer(8),
    PCATransformer(7),
    PCATransformer(6),
    PCATransformer(5),
    PCATransformer(4),
]

In [ ]:
# 25min
model_list_1 = [
    ('sv1', SVC(C=1.0, kernel='rbf', gamma='scale', random_state=42)),
    ('kn1', KNeighborsClassifier(n_neighbors=5)),
]

model_list_2 = [
    ('rf2', RandomForestClassifier(n_estimators=25, max_depth=2, random_state=42)),
    ('gb2', GradientBoostingClassifier(n_estimators=25, max_depth=2, random_state=42)),
    ('sv2', SVC(C=1.0, kernel='rbf', gamma='scale', random_state=42)),
    ('kn2', KNeighborsClassifier(n_neighbors=5, )),
    ('dt2', DecisionTreeClassifier(max_depth=2, random_state=42)),
]

all_model_combinations = list(product(model_list_1, model_list_2))
formatted_model_combinations = [[(name1, model1), (name2, model2)] for ((name1, model1), (name2, model2)) in all_model_combinations]

for transformer in newTransformers:
    stack = StackingClassifier(estimators=[], final_estimator=LogisticRegression())
    pipeline = Pipeline([
        ('preprocessing', transformer),
        ('stack', stack)
    ])

    param_grid = {
        'stack__estimators':formatted_model_combinations
    }

    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        n_jobs=n_jobs,
        verbose=verbose,
        cv=cv,
        scoring='balanced_accuracy'
        )

    grid_search.fit(X_train, y_train)

    cs_results = pd.DataFrame(grid_search.cv_results_)
    save_data_to_csv(cs_results, f"../Wyniki/ModelSearch/Etap2/{type(transformer).__name__}.csv")

In [ ]:
analisys2_PCA4Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap2/PCA4Transformer.csv')
analisys2_PCA5Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap2/PCA5Transformer.csv')
analisys2_PCA6Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap2/PCA6Transformer.csv')
analisys2_PCA7Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap2/PCA7Transformer.csv')
analisys2_PCA8Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap2/PCA8Transformer.csv')
analisys2_PCA9Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap2/PCA9Transformer.csv')
analisys2_PCA03Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap2/PCA03Transformer.csv')
analisys2_PCA04Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap2/PCA04Transformer.csv')
analisys2_RFCTransformer = pd.read_csv('../Wyniki/ModelSearch/Etap2/RFCSelector.csv')

In [ ]:
transformers = ["RFCTransformer", "PCA04Transformer", "PCA03Transformer",
                "PCA9Transformer", "PCA8Transformer", "PCA7Transformer",
                "PCA6Transformer", "PCA5Transformer", "PCA4Transformer"]

analisys_list = [analisys2_RFCTransformer, analisys2_PCA04Transformer, analisys2_PCA03Transformer,
                 analisys2_PCA9Transformer, analisys2_PCA8Transformer, analisys2_PCA7Transformer,
                 analisys2_PCA6Transformer, analisys2_PCA5Transformer, analisys2_PCA4Transformer]

print_results(transformers, analisys_list)

Można zauważyć, że najlepsze wyniki oraz średnio najlepsze wyniki uzyskano dla PCA5 oraz PCA7. Dlatego w dalszych rozważaniach tylko te metody będą brane pod uwagę

In [ ]:
print_best_results(analisys2_PCA03Transformer, "PCA03Transformer")
print_best_results(analisys2_PCA04Transformer, "PCA04Transformer")
print_best_results(analisys2_RFCTransformer, "RFCTransformer")
print_best_results(analisys2_PCA5Transformer, "PCA5Transformer")
print_best_results(analisys2_PCA7Transformer, "PCA7Transformer")

Wyniki nie różnią się znacząco od poprzednich. Dla
Na podstawie powyższych wyników zdecydowano się tunować następujące algorytmy: 
- KNeighborsClassifier z SVC
- KNeighborsClassifier z RandomForestClassifier
- KNeighborsClassifier z GradientBoostingClassifier
- KNeighborsClassifier z DecisionTreeClassifier

i metody preprocessingu PCA5 i PCA7 
Poniżej wykonano tunowanie omawianych modeli. 

Spodziewane wyniki to przewaga modelu KNeighborsClassifier z SVC po tuningu nad resztą algorytmów. Przewaga PCA5 nad PCA7 we wszystkich przypadkach.

# Etap 3

In [ ]:
finalTransformers = [
    PCATransformer(5),
    PCATransformer(7),
]

In [ ]:
# 1h 15min 
for transformer in finalTransformers:
    stack = StackingClassifier(estimators=[ 
        ('kn1', KNeighborsClassifier()),
        ('rf1', RandomForestClassifier(random_state=random_state)),
        ], final_estimator=LogisticRegression())
    pipeline = Pipeline([
        ('preprocessing', transformer),
        ('model', stack)
    ])

    param_dist = {
    'model__rf1__n_estimators': np.linspace(n_estimators_lower_rf, n_estimators_upper_rf, n_estimators_upper_rf - n_estimators_lower_rf).astype(int),
    'model__rf1__max_depth': np.linspace(max_depth_lower_rf, max_depth_upper_rf, max_depth_upper_rf- max_depth_lower_rf).astype(int),
    'model__rf1__min_samples_split': np.linspace(min_samples_split_lower_rf, min_samples_split_upper_rf, min_samples_split_upper_rf - min_samples_split_lower_rf).astype(int),
    'model__rf1__min_samples_leaf': np.linspace(min_samples_leaf_lower_rf, min_samples_leaf_upper_rf, min_samples_leaf_upper_rf - min_samples_leaf_lower_rf).astype(int),
    'model__rf1__max_features': max_features_rf,
    'model__kn1__n_neighbors': np.linspace(n_neighbors_lower, n_neighbors_upper, n_neighbors_upper - n_neighbors_lower).astype(int),
    'model__kn1__weights': weights,
    'model__kn1__p': p
    }

    random_search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_dist,
        n_jobs=n_jobs,
        n_iter=n_iter_randomSearch_rf,
        verbose=verbose,
        random_state=random_state,
        cv=cv,
        scoring='balanced_accuracy'
        )

    random_search.fit(X_train, y_train)

    cs_results = pd.DataFrame(random_search.cv_results_)
    save_data_to_csv(cs_results, f"../Wyniki/ModelSearch/Etap3/KNRF{type(transformer).__name__}.csv")

In [ ]:
# 1h 10min 
for transformer in finalTransformers:
    stack = StackingClassifier(estimators=[ 
        ('kn1', KNeighborsClassifier()),
        ('sv1', SVC(random_state=random_state)),
        ], final_estimator=LogisticRegression())
    pipeline = Pipeline([
        ('preprocessing', transformer),
        ('model', stack)
    ])

    param_dist = {
    'model__sv1__C': np.linspace(cost_lower_sv, cost_upper_sv, numberOfPoints).astype(float),
    'model__sv1__kernel': kernel_sv,
    'model__sv1__gamma':  np.linspace(gamma_lower_sv, gamma_upper_sv, numberOfPoints).astype(float),
    'model__kn1__n_neighbors': np.linspace(n_neighbors_lower, n_neighbors_upper, n_neighbors_upper - n_neighbors_lower).astype(int),
    'model__kn1__weights': weights,
    'model__kn1__p': p
    }

    random_search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_dist,
        n_jobs=n_jobs,
        n_iter=n_iter_randomSearch_sv,
        verbose=verbose,
        random_state=random_state,
        cv=cv,
        scoring='balanced_accuracy'
        )

    random_search.fit(X_train, y_train)

    cs_results = pd.DataFrame(random_search.cv_results_)
    save_data_to_csv(cs_results, f"../Wyniki/ModelSearch/Etap3/KNSV{type(transformer).__name__}.csv")

In [ ]:
# 1h 45min 
for transformer in finalTransformers:
    stack = StackingClassifier(estimators=[ 
        ('kn1', KNeighborsClassifier()),
        ('gb1', GradientBoostingClassifier(random_state=random_state)),
        ], final_estimator=LogisticRegression())
    pipeline = Pipeline([
        ('preprocessing', transformer),
        ('model', stack)
    ])

    param_dist = {
    'model__gb1__n_estimators': np.linspace(n_estimators_lower_gb, n_estimators_upper_gb, n_estimators_upper_gb - n_estimators_lower_gb).astype(int),
    'model__gb1__max_depth': np.linspace(max_depth_lower_gb, max_depth_upper_gb, max_depth_upper_gb- max_depth_lower_gb).astype(int),
    'model__gb1__min_samples_split': np.linspace(min_samples_split_lower_gb, min_samples_split_upper_gb, min_samples_split_upper_gb - min_samples_split_lower_gb).astype(int),
    'model__gb1__min_samples_leaf': np.linspace(min_samples_leaf_lower_gb, min_samples_leaf_upper_gb, min_samples_leaf_upper_gb - min_samples_leaf_lower_gb).astype(int),
    'model__gb1__learning_rate': np.linspace(learning_rate_lower_gb, learning_rate_upper_gb, numberOfPoints).astype(float),
    'model__gb1__subsample': np.linspace(subsample_lower_gb, subsample_upper_gb, numberOfPoints).astype(float),
    'model__kn1__n_neighbors': np.linspace(n_neighbors_lower, n_neighbors_upper, n_neighbors_upper - n_neighbors_lower).astype(int),
    'model__kn1__weights': weights,
    'model__kn1__p': p
    }

    random_search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_dist,
        n_jobs=n_jobs,
        n_iter=n_iter_randomSearch_gb,
        verbose=verbose,
        random_state=random_state,
        cv=cv,
        scoring='balanced_accuracy'
        )

    random_search.fit(X_train, y_train)

    cs_results = pd.DataFrame(random_search.cv_results_)
    save_data_to_csv(cs_results, f"../Wyniki/ModelSearch/Etap3/KNGB{type(transformer).__name__}.csv")

In [ ]:
# 1h 30min 
for transformer in finalTransformers:
    stack = StackingClassifier(estimators=[ 
        ('kn1', KNeighborsClassifier()),
        ('dt1', DecisionTreeClassifier(random_state=random_state)),
        ], final_estimator=LogisticRegression())
    pipeline = Pipeline([
        ('preprocessing', transformer),
        ('model', stack)
    ])

    param_dist = {
    'model__dt1__max_depth': np.linspace(max_depth_lower_dt, max_depth_upper_dt, max_depth_upper_dt- max_depth_lower_dt).astype(int),
    'model__dt1__min_samples_split': np.linspace(min_samples_split_lower_dt, min_samples_split_upper_dt, min_samples_split_upper_dt - min_samples_split_lower_dt).astype(int),
    'model__dt1__min_samples_leaf': np.linspace(min_samples_leaf_lower_dt, min_samples_leaf_upper_dt, min_samples_leaf_upper_dt - min_samples_leaf_lower_dt).astype(int),
    'model__dt1__max_features': max_features_dt,
    'model__kn1__n_neighbors': np.linspace(n_neighbors_lower, n_neighbors_upper, n_neighbors_upper - n_neighbors_lower).astype(int),
    'model__kn1__weights': weights,
    'model__kn1__p': p
    }

    random_search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_dist,
        n_jobs=n_jobs,
        n_iter=n_iter_randomSearch_dt,
        verbose=verbose,
        random_state=random_state,
        cv=cv,
        scoring='balanced_accuracy'
        )

    random_search.fit(X_train, y_train)

    cs_results = pd.DataFrame(random_search.cv_results_)
    save_data_to_csv(cs_results, f"../Wyniki/ModelSearch/Etap3/KNDT{type(transformer).__name__}.csv")

In [ ]:
analisys3_KN_DT_PCA5Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap3/KNDTPCA5Transformer.csv')
analisys3_KN_GB_PCA5Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap3/KNGBPCA5Transformer.csv')
analisys3_KN_RF_PCA5Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap3/KNRFPCA5Transformer.csv')
analisys3_KN_SV_PCA5Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap3/KNSVPCA5Transformer.csv')
analisys3_KN_DT_PCA7Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap3/KNDTPCA7Transformer.csv')
analisys3_KN_GB_PCA7Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap3/KNGBPCA7Transformer.csv')
analisys3_KN_RF_PCA7Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap3/KNRFPCA7Transformer.csv')
analisys3_KN_SV_PCA7Transformer = pd.read_csv('../Wyniki/ModelSearch/Etap3/KNSVPCA7Transformer.csv')

In [ ]:
transformers = ["KN_DT_PCA5Transformer", "KN_GB_PCA5Transformer", "KN_RF_PCA5Transformer",
                "KN_SV_PCA5Transformer", "KN_DT_PCA7Transformer", "KN_GB_PCA7Transformer",
                "KN_RF_PCA7Transformer", "KN_SV_PCA7Transformer"]

analisys_list = [analisys3_KN_DT_PCA5Transformer, analisys3_KN_GB_PCA5Transformer, analisys3_KN_RF_PCA5Transformer,
                 analisys3_KN_SV_PCA5Transformer, analisys3_KN_DT_PCA7Transformer, analisys3_KN_GB_PCA7Transformer,
                 analisys3_KN_RF_PCA7Transformer, analisys3_KN_SV_PCA7Transformer]

print_results(transformers, analisys_list)

In [ ]:
print(searchMaxRowByColumn(analisys3_KN_GB_PCA5Transformer,'mean_test_score'))

Można zauważyć, że przewidywania co do przewagi modelu z SVM okazały się błędne, a najlepszym modelem okazał się stack zawierający KNeighborsClassifier i GradientBoostingClassifier

# Etap 4

Spróbujmy uzyskać jeszcze lepsze wyniki na najlepszym stacku poprzez wylosowanie w metodzie randomSearch 5 razy więcej punktów niż poprzednio.

In [ ]:
# 3h 45min 
stack = StackingClassifier(estimators=[ 
    ('kn1', KNeighborsClassifier()),
    ('gb1', GradientBoostingClassifier(random_state=random_state)),
    ], final_estimator=LogisticRegression())
pipeline = Pipeline([
        ('preprocessing', PCA5Transformer()),
        ('model', stack)
    ])

param_dist = {
    'model__gb1__n_estimators': np.linspace(n_estimators_lower_gb, n_estimators_upper_gb, n_estimators_upper_gb - n_estimators_lower_gb).astype(int),
    'model__gb1__max_depth': np.linspace(max_depth_lower_gb, max_depth_upper_gb, max_depth_upper_gb- max_depth_lower_gb).astype(int),
    'model__gb1__min_samples_split': np.linspace(min_samples_split_lower_gb, min_samples_split_upper_gb, min_samples_split_upper_gb - min_samples_split_lower_gb).astype(int),
    'model__gb1__min_samples_leaf': np.linspace(min_samples_leaf_lower_gb, min_samples_leaf_upper_gb, min_samples_leaf_upper_gb - min_samples_leaf_lower_gb).astype(int),
    'model__gb1__learning_rate': np.linspace(learning_rate_lower_gb, learning_rate_upper_gb, numberOfPoints).astype(float),
    'model__gb1__subsample': np.linspace(subsample_lower_gb, subsample_upper_gb, numberOfPoints).astype(float),
    'model__kn1__n_neighbors': np.linspace(n_neighbors_lower, n_neighbors_upper, n_neighbors_upper - n_neighbors_lower).astype(int),
    'model__kn1__weights': weights,
    'model__kn1__p': p
    }

random_search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_dist,
        n_jobs=n_jobs,
        n_iter=n_iter_randomSearch_final,
        verbose=verbose,
        random_state=random_state,
        cv=cv,
        scoring='balanced_accuracy'
        )

random_search.fit(X_train, y_train)

cs_results = pd.DataFrame(random_search.cv_results_)
save_data_to_csv(cs_results, f"../Wyniki/ModelSearch/Etap4/FianlModel.csv")

In [ ]:
analisys_final_best = pd.read_csv('../Wyniki/ModelSearch/Etap4/FianlModel.csv')

In [ ]:
print("\nNajlepszy znaleziony model:\n")
print(searchMaxRowByColumn(analisys_final_best,'mean_test_score'))

Niestety nie widać aż tak dużej poprawy względem tego co uzyskano pod koniec etapu 3 

# Analiza końcowa i trening finałowych modeli

Trening następujących stacków do analizy końcowej: 
- KNeighborsClassifier z SVC z 3 etapu 
- KNeighborsClassifier z RandomForestClassifier z 3 etapu 
- KNeighborsClassifier z GradientBoostingClassifier z 4 etapu 
- KNeighborsClassifier z DecisionTreeClassifier z 3 etapu 

In [ ]:
# best result: 0.88948, sprawdzarka: 0.9667
stack = StackingClassifier(estimators=[ ('dt1', DecisionTreeClassifier(min_samples_split=2, min_samples_leaf=3, max_features='sqrt', max_depth=6, random_state=42)),
    ('kn1', KNeighborsClassifier(n_neighbors=4, p=2 , weights='distance'))], final_estimator=LogisticRegression())
pipeline = Pipeline([
        ('preprocessing', PCA5Transformer()),
        ('stack', stack)
    ])
pipeline.fit(X_train, y_train)
proba = pipeline.predict_proba(X_test)

proba_dt = pd.DataFrame(proba).iloc[:,1:]

save_prediction_to_file(proba_dt, '../Wyniki/Predykcje/MANUAL/KN_DT_')

In [ ]:
# best result: 0.895988, sprawdzarka: 0.9333
stack = StackingClassifier(estimators=[ ('gb1', GradientBoostingClassifier( subsample=0.833333333333333, n_estimators=91,  min_samples_split=8, min_samples_leaf=3, learning_rate=0.21875, max_depth=8, random_state=42)),
    ('kn1', KNeighborsClassifier(n_neighbors=3, p=2 , weights='distance'))], final_estimator=LogisticRegression())
pipeline = Pipeline([
        ('preprocessing', PCA5Transformer()),
        ('stack', stack)
    ])
pipeline.fit(X_train, y_train)
proba = pipeline.predict_proba(X_test)

proba_gb = pd.DataFrame(proba).iloc[:,1:]

save_prediction_to_file(proba_gb, '../Wyniki/Predykcje/MANUAL/KN_GB_')

In [ ]:
# best result: 0.88849, sprawdzarka: 0.9333
stack = StackingClassifier(estimators=[ ('rf1', RandomForestClassifier( max_features='sqrt', n_estimators=36,  min_samples_split=3, min_samples_leaf=5, max_depth=8, random_state=42)),
    ('kn1', KNeighborsClassifier(n_neighbors=7, p=2 , weights='distance'))], final_estimator=LogisticRegression())
pipeline = Pipeline([
        ('preprocessing', PCA5Transformer()),
        ('stack', stack)
    ])
pipeline.fit(X_train, y_train)
proba = pipeline.predict_proba(X_test)

proba_rf = pd.DataFrame(proba).iloc[:,1:]

save_prediction_to_file(proba_rf, '../Wyniki/Predykcje/MANUAL/KN_RF_')

In [ ]:
# best result: 0.888488, sprawdzarka: nie było czasu aby sprawdzić, ale ten wynik na sprawdzarce by nic nie zmienił 
stack = StackingClassifier(estimators=[ ('sv1', SVC(kernel='rbf', gamma=6.5, C=1.25875, random_state=42)),
    ('kn1', KNeighborsClassifier(n_neighbors=4, p=2 , weights='distance'))], final_estimator=LogisticRegression())
pipeline = Pipeline([
        ('preprocessing', PCA5Transformer()),
        ('stack', stack)
    ])
pipeline.fit(X_train, y_train)
proba = pipeline.predict_proba(X_test)

proba_sv = pd.DataFrame(proba).iloc[:,1:]

save_prediction_to_file(proba_sv, '../Wyniki/Predykcje/MANUAL/KN_SV_')

Poniżej widczony jest rozkład zmiennej odpowiedzi w zależności od modelu. Najbardziej symetryczny rozkład występuję dla proba_gb. Dlatego też wybieramy ten model jako nasz najlepszy, pomimo tego, że w sprawdzarce uzyskał 2 najlepszy wynik  

In [ ]:
plt.figure(figsize=(12, 8))
proba_dt.plot(kind='hist', bins=50, alpha=0.5)
plt.title('Zmienna odpowiedzi dla stacka zawierającego algorytm DecisionTreeClassifier')
plt.xlabel('Wartość')
plt.ylabel('Liczba Obserwacji')
plt.savefig('../Wyniki/Wykresy/' + "proba_dt" + '.jpg', format='jpeg', dpi=300, bbox_inches='tight')
plt.show()

plt.figure(figsize=(12, 8))
proba_gb.plot(kind='hist', bins=50, alpha=0.5)
plt.title('Zmienna odpowiedzi dla stacka zawierającego algorytm GradientBoostingClassifier')
plt.xlabel('Wartość')
plt.ylabel('Liczba Obserwacji')
plt.savefig('../Wyniki/Wykresy/' + "proba_gb" + '.jpg', format='jpeg', dpi=300, bbox_inches='tight')
plt.show()

plt.figure(figsize=(12, 8))
proba_rf.plot(kind='hist', bins=50, alpha=0.5)
plt.title('Zmienna odpowiedzi dla stacka zawierającego algorytm RandomForestClassifier')
plt.xlabel('Wartość')
plt.ylabel('Liczba Obserwacji')
plt.savefig('../Wyniki/Wykresy/' + "proba_rf" + '.jpg', format='jpeg', dpi=300, bbox_inches='tight')
plt.show()

plt.figure(figsize=(12, 8))
proba_sv.plot(kind='hist', bins=50, alpha=0.5)
plt.title('Zmienna odpowiedzi dla stacka zawierającego algorytm SVM')
plt.xlabel('Wartość')
plt.ylabel('Liczba Obserwacji')
plt.savefig('../Wyniki/Wykresy/' + "proba_sv" + '.jpg', format='jpeg', dpi=300, bbox_inches='tight')
plt.show()